### Simple neural network
<font size = 3> Simple neural network tasked with identifying AGN with jets from point sources with close to 100% accuracy. 

In [1]:
import tensorflow as tf
import numpy as np
import os
from astropy.io import fits 
from astropy import stats
from scipy.ndimage import rotate
import matplotlib.pyplot as plt

In [2]:
# These functions have since been updated and streamlined and are included in the building training set file. 

def get_clipped_images(filepath,xpix,ypix,sigma):
    '''
    Put FITS data from desired folder into a 3D array
    sigma = how many sigmas from the median background value to sigma clip the data to
    '''
    newpath = filepath.replace(os.sep, '/')
    dirs = os.listdir(newpath)
    n = len(dirs)
    data = np.empty(shape=(n,xpix,ypix),dtype=np.float64)
    for i in range(n):
        fullpath = '{}/{}'.format(newpath,dirs[i])
        d = fits.getdata(fullpath, ext=0)
        d[np.isnan(d)] = 0
        _,median,std = stats.sigma_clipped_stats(d, sigma=sigma)
        d[d<median+sigma*std] = median+sigma*std
        data[i,:,:] = d
    return data

def augment_data(data,size,xpix,ypix):
    '''
    Augment the data (3D array of images) by flipping and rotating the images.
    Size = upper bound on the final number of images 
    (actual_size can be much less depending on size/data_size multiples)
    '''
    rotations = size//len(data) # rotations per image
    angles = np.linspace(0, 360, rotations)
    act_size = rotations*len(data)
    training_set = np.empty((act_size, xpix, ypix))
    for i in range(len(data)):
        for j in range(len(angles)):
            if j % 2 == 0: training_set[i*len(angles)+j,:,:] = rotate(np.fliplr(data[i,:,:]), angles[j], reshape=False)
            else: training_set[i*len(angles)+j,:,:] = rotate(data[i,:,:], angles[j], reshape=False)
    return training_set

def train_test(data,percentage):
    '''
    Combines data sets in one 3D array, with a different label for each data set.
    Then randomly shuffles the data and splits into training and test sets.
    data = list 3D arrays containing desired data sets
    per = fraction of data to be in training set
    returns: train and test data (each a tupple containing the data and corresponding labels)
    '''
    d = np.concatenate(data,axis=0)
    n_images = len(d)
    labels = np.empty(n_images)
    i = 0
    for n in range(len(data)):
        labels[i:i+len(data[n])] = n
        i = len(data[n])
    rand_ind = np.random.permutation(range(n_images))
    d, labels = d[rand_ind], labels[rand_ind]
    n_train = np.int(np.round(n_images*percentage))
    train = (d[:n_train], labels[:n_train])
    test = (d[n_train:], labels[n_train:])
    return train, test

In [3]:
# Number of x pixels and y pixels in each image (must be the same for all images)
xpix, ypix = 83, 83

# Directories with the FITS data
agn_path = r'C:\Users\Cerys\Documents\Physics\Y4 Project\Data Preparation\Radio Zoo Images'
ps_path = r'C:\Users\Cerys\Documents\Physics\Y4 Project\Data Preparation\Stars'

# Get FITs images and augment data sets
agn_data = get_clipped_images(agn_path,xpix,ypix,sigma=3)
ps_data = get_clipped_images(ps_path,xpix,ypix,sigma=1)
agn_augment = augment_data(agn_data,10000,xpix,ypix)
ps_augment = augment_data(ps_data,10000,xpix,ypix)

data = (agn_augment, ps_augment) # list of all data sets to be used

In [4]:
train, test = train_test(data,0.2) # training and test data, each in a tuple with data label array

In [5]:
# Neural network
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(xpix, ypix)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
    
# Compile the network
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# Perform fit
model.fit(*train, epochs=10)
test_loss, test_acc = model.evaluate(*test, verbose=2)
print('\nTest accuracy:', test_acc)

Train on 3482 samples
Epoch 1/10
3482/3482 [==============================] - 3s 893us/sample - loss: 0.7643 - accuracy: 0.9024
Epoch 2/10
3482/3482 [==============================] - 2s 679us/sample - loss: 0.0353 - accuracy: 1.0000
Epoch 3/10
3482/3482 [==============================] - 2s 712us/sample - loss: 0.0102 - accuracy: 1.0000
Epoch 4/10
3482/3482 [==============================] - 2s 667us/sample - loss: 0.0052 - accuracy: 1.0000
Epoch 5/10
3482/3482 [==============================] - 2s 650us/sample - loss: 0.0033 - accuracy: 1.0000
Epoch 6/10
3482/3482 [==============================] - 2s 662us/sample - loss: 0.0023 - accuracy: 1.0000
Epoch 7/10
3482/3482 [==============================] - 2s 626us/sample - loss: 0.0017 - accuracy: 1.0000
Epoch 8/10
3482/3482 [==============================] - 2s 618us/sample - loss: 0.0012 - accuracy: 1.0000
Epoch 9/10
3482/3482 [==============================] - 2s 634us/sample - loss: 9.8631e-04 - accuracy: 1.0000
Epoch 10/10
3482/348

Above shows can achieve 100% accuracy between point sources and AGN with jets even with a simple neural net.